In [ ]:
import tensorflow as tf
import numpy as np
import keras
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [ ]:
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    data_augmentation.add(RandomZoom(0.2))
    return data_augmentation

def convolution_block(X, f, filters, s = 2, training = True):
    F1, F2, F3 = filters
    X_shortcut = X
    #First Block
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
    X = tf.keras.layers.Activation('relu')(X)
    #Second Block
    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
    X = tf.keras.layers.Activation('relu')(X)
    #Third Block
    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
    #Third Block
    X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid')(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis=3)(X_shortcut, training=training)
    #Final Block
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    return X

def identity_block(X, f, filters, training=True):
    F1, F2, F3 = filters
    X_shortcut = X
    # First Block
    X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
    X = tf.keras.layers.Activation('relu')(X)
    # Second Block
    X = tf.keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
    X = tf.keras.layers.Activation('relu')(X)
    # Third Block
    X = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization(axis=3)(X, training=training)
    # Final Block
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    return X



In [ ]:
#We get the images from the Intel Dataset
batch_size = 20
IMG_SIZE = (150,150)
directory = r'Intel_dataset/seg_train'
train_data = image_dataset_from_directory(directory,
                                          shuffle=True,
                                          batch_size=batch_size,
                                          image_size=IMG_SIZE,
                                          validation_split=0.1,
                                          subset='training',
                                          seed=42,
                                          label_mode='categorical')

test_data = image_dataset_from_directory(directory,
                                              shuffle=True,
                                              batch_size=batch_size,
                                              image_size=IMG_SIZE,
                                              validation_split=0.1,
                                              subset='validation',
                                              seed=42,
                                              label_mode='categorical')

#Autotune allow us to prevent bottleneck when we are reading from disk
Autotune = tf.data.experimental.AUTOTUNE
train_data = train_data.prefetch(buffer_size = Autotune)

X_input = tf.keras.layers.Input((150,150,3))
X = tf.keras.layers.ZeroPadding2D((3,3))(X_input)
X = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2))(X)
X = tf.keras.layers.BatchNormalization(axis=3)(X)
X = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2))(X)
X = convolution_block(X, f = 3, filters=[64,64,256], s = 1)
X = identity_block(X, 3, [64,64,256])
X = identity_block(X, 3, [64,64,256])
X = convolution_block(X, f = 3, filters = [128,128,512], s = 2)
X = identity_block(X, 3, [128,128,512])
X = identity_block(X, 3, [128,128,512])
X = identity_block(X, 3, [128,128,512])
X = convolution_block(X, f = 3, filters= [256,256,1024], s=2)
X = identity_block(X, 3, [256,256,1024])
X = identity_block(X, 3, [256,256,1024])
X = identity_block(X, 3, [256,256,1024])
X = identity_block(X, 3, [256,256,1024])
X = identity_block(X, 3, [256,256,1024])
X = convolution_block(X, f = 3, filters= [512,512,2048], s=2)
X = identity_block(X, 3, [512,512,2048])
X = identity_block(X, 3, [512,512,2048])
X = tf.keras.layers.AveragePooling2D(pool_size=(2,2))(X)
X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dense(4096, activation='relu')(X)
X = tf.keras.layers.Dense(6, activation='softmax')(X)

model = Model(inputs= X_input, outputs = X)
plot_model(model)
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_data, validation_data=test_data, epochs=5, verbose=2)

model.save('intel_model_tf.h5')